In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import models, layers, activations, initializers, losses, optimizers, metrics
from keras.models import Sequential, Model
from keras.layers import Dense, Input, LeakyReLU, Dropout, BatchNormalization
from sklearn.preprocessing import MinMaxScaler, StandardScaler

Using TensorFlow backend.
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\student\Anaconda3\lib\site-packages\tensorflow\python\framework\d

In [47]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    length = len(sequence)
    col_length = len(sequence[0])
    for i in range(length):
        end_ix = i + n_steps
        if end_ix > length-1:
            break
        seq_x, seq_y = sequence[i:end_ix, :], sequence[end_ix, col_length-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [48]:
data = pd.read_csv('data/samsung.csv', encoding='euc-kr')
data = data.sort_values(['일자'], ascending=[True])
data = data.iloc[:,1:].copy()

data = pd.concat([data.iloc[:,:3], data.iloc[:,4], data.iloc[:,3]], axis=1)

for i in data.columns:
    data[i] = data[i].apply(lambda x: x.replace(",","",-1))

for i in data.columns:
    data[i] = data[i].astype(int)

data = np.array(data)

In [49]:
n_steps = 5
x_data, y_data = split_sequence(data, n_steps)

In [50]:
x_data = x_data.reshape((x_data.shape[0], -1))
scaler = StandardScaler()
scaler.fit(x_data)
x_data = scaler.transform(x_data)
x_data.shape

(421, 25)

In [51]:
final_predict_data = x_data[-1:,:].copy()

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, train_size=0.9, random_state=66, shuffle=True
)

In [52]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(x_data.shape[1],)))
model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='linear'))

# # 훈련
# model.compile(loss='mse', optimizer='adam', metrics=['acc'])
# model.fit(x, y, epochs=100, batch_size=1, validation_split=0.1)

In [53]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=50, mode='auto')
model.compile(loss='mse', optimizer='adam', metrics=['acc'])
model.fit(x_train, y_train, epochs=1000, validation_split=0.1,
         callbacks=[early_stopping])

Train on 340 samples, validate on 38 samples
Epoch 1/1000
340/340 [==============================] - 0s 1ms/step - loss: 2201442418.4471 - acc: 0.0000e+00 - val_loss: 2197402462.3158 - val_acc: 0.0000e+00
Epoch 2/1000
340/340 [==============================] - 0s 91us/step - loss: 2191488816.1882 - acc: 0.0000e+00 - val_loss: 2167539947.7895 - val_acc: 0.0000e+00
Epoch 3/1000
340/340 [==============================] - 0s 94us/step - loss: 2103533277.3647 - acc: 0.0000e+00 - val_loss: 1930050674.5263 - val_acc: 0.0000e+00
Epoch 4/1000
340/340 [==============================] - 0s 94us/step - loss: 1570679387.8588 - acc: 0.0000e+00 - val_loss: 968351959.5789 - val_acc: 0.0000e+00
Epoch 5/1000
340/340 [==============================] - 0s 94us/step - loss: 898655074.6353 - acc: 0.0000e+00 - val_loss: 820764485.0526 - val_acc: 0.0000e+00
Epoch 6/1000
340/340 [==============================] - 0s 94us/step - loss: 706873923.0118 - acc: 0.0000e+00 - val_loss: 740719575.5789 - val_acc: 0.0000

340/340 [==============================] - 0s 97us/step - loss: 4975515.1426 - acc: 0.0000e+00 - val_loss: 5594913.8947 - val_acc: 0.0000e+00
Epoch 53/1000
340/340 [==============================] - 0s 100us/step - loss: 4711473.5765 - acc: 0.0000e+00 - val_loss: 7472385.5395 - val_acc: 0.0000e+00
Epoch 54/1000
340/340 [==============================] - 0s 97us/step - loss: 5496394.0971 - acc: 0.0000e+00 - val_loss: 5028903.5461 - val_acc: 0.0000e+00
Epoch 55/1000
340/340 [==============================] - 0s 103us/step - loss: 4421563.8588 - acc: 0.0000e+00 - val_loss: 4637537.3553 - val_acc: 0.0000e+00
Epoch 56/1000
340/340 [==============================] - 0s 97us/step - loss: 3823157.3588 - acc: 0.0000e+00 - val_loss: 4883544.3158 - val_acc: 0.0000e+00
Epoch 57/1000
340/340 [==============================] - 0s 91us/step - loss: 3468469.3118 - acc: 0.0000e+00 - val_loss: 4708648.6053 - val_acc: 0.0000e+00
Epoch 58/1000
340/340 [==============================] - 0s 82us/step - loss

Epoch 105/1000
340/340 [==============================] - 0s 82us/step - loss: 825455.3449 - acc: 0.0000e+00 - val_loss: 1223428.3586 - val_acc: 0.0000e+00
Epoch 106/1000
340/340 [==============================] - 0s 83us/step - loss: 931357.1478 - acc: 0.0000e+00 - val_loss: 1776673.9934 - val_acc: 0.0000e+00
Epoch 107/1000
340/340 [==============================] - 0s 88us/step - loss: 999899.5292 - acc: 0.0059 - val_loss: 1342013.5855 - val_acc: 0.0000e+00
Epoch 108/1000
340/340 [==============================] - 0s 91us/step - loss: 938277.8478 - acc: 0.0000e+00 - val_loss: 1178540.4507 - val_acc: 0.0000e+00
Epoch 109/1000
340/340 [==============================] - 0s 91us/step - loss: 743134.1029 - acc: 0.0000e+00 - val_loss: 1181100.0526 - val_acc: 0.0000e+00
Epoch 110/1000
340/340 [==============================] - 0s 91us/step - loss: 876362.3588 - acc: 0.0000e+00 - val_loss: 1117157.3553 - val_acc: 0.0000e+00
Epoch 111/1000
340/340 [==============================] - 0s 85us/st

Epoch 158/1000
340/340 [==============================] - 0s 91us/step - loss: 505818.8625 - acc: 0.0000e+00 - val_loss: 1031139.4211 - val_acc: 0.0000e+00
Epoch 159/1000
340/340 [==============================] - 0s 91us/step - loss: 557733.3456 - acc: 0.0000e+00 - val_loss: 1041606.9901 - val_acc: 0.0000e+00
Epoch 160/1000
340/340 [==============================] - 0s 85us/step - loss: 681625.9246 - acc: 0.0000e+00 - val_loss: 1178236.3454 - val_acc: 0.0000e+00
Epoch 161/1000
340/340 [==============================] - 0s 88us/step - loss: 552503.0522 - acc: 0.0000e+00 - val_loss: 1120625.5888 - val_acc: 0.0000e+00
Epoch 162/1000
340/340 [==============================] - 0s 88us/step - loss: 557895.2301 - acc: 0.0000e+00 - val_loss: 966821.2796 - val_acc: 0.0000e+00
Epoch 163/1000
340/340 [==============================] - 0s 85us/step - loss: 555895.9801 - acc: 0.0029 - val_loss: 1445713.7204 - val_acc: 0.0000e+00
Epoch 164/1000
340/340 [==============================] - 0s 82us/ste

340/340 [==============================] - 0s 86us/step - loss: 583574.0272 - acc: 0.0000e+00 - val_loss: 956156.4276 - val_acc: 0.0000e+00
Epoch 212/1000
340/340 [==============================] - 0s 85us/step - loss: 508702.0029 - acc: 0.0088 - val_loss: 992445.4737 - val_acc: 0.0000e+00
Epoch 213/1000
340/340 [==============================] - 0s 94us/step - loss: 417378.6932 - acc: 0.0000e+00 - val_loss: 1072074.5757 - val_acc: 0.0000e+00
Epoch 214/1000
340/340 [==============================] - 0s 100us/step - loss: 394215.8272 - acc: 0.0000e+00 - val_loss: 950155.7072 - val_acc: 0.0000e+00
Epoch 215/1000
340/340 [==============================] - 0s 88us/step - loss: 417075.9272 - acc: 0.0000e+00 - val_loss: 1086847.9112 - val_acc: 0.0000e+00
Epoch 216/1000
340/340 [==============================] - 0s 94us/step - loss: 414404.3507 - acc: 0.0000e+00 - val_loss: 1079463.8947 - val_acc: 0.0000e+00
Epoch 217/1000
340/340 [==============================] - 0s 88us/step - loss: 451166

Epoch 264/1000
340/340 [==============================] - 0s 97us/step - loss: 527089.9528 - acc: 0.0000e+00 - val_loss: 1186154.0921 - val_acc: 0.0000e+00
Epoch 265/1000
340/340 [==============================] - 0s 93us/step - loss: 423999.4265 - acc: 0.0000e+00 - val_loss: 1034354.7336 - val_acc: 0.0000e+00
Epoch 266/1000
340/340 [==============================] - 0s 91us/step - loss: 378838.0743 - acc: 0.0000e+00 - val_loss: 1008055.0230 - val_acc: 0.0000e+00
Epoch 267/1000
340/340 [==============================] - 0s 91us/step - loss: 369523.7941 - acc: 0.0000e+00 - val_loss: 1570643.6151 - val_acc: 0.0000e+00
Epoch 268/1000
340/340 [==============================] - 0s 88us/step - loss: 590901.4081 - acc: 0.0000e+00 - val_loss: 1119615.3092 - val_acc: 0.0000e+00
Epoch 269/1000
340/340 [==============================] - 0s 88us/step - loss: 583104.1577 - acc: 0.0000e+00 - val_loss: 2006223.8520 - val_acc: 0.0000e+00
Epoch 270/1000
340/340 [==============================] - 0s 91u

Epoch 317/1000
340/340 [==============================] - 0s 76us/step - loss: 468065.1243 - acc: 0.0029 - val_loss: 1118252.8355 - val_acc: 0.0000e+00
Epoch 318/1000
340/340 [==============================] - 0s 74us/step - loss: 353302.1147 - acc: 0.0000e+00 - val_loss: 1030472.6053 - val_acc: 0.0000e+00
Epoch 319/1000
340/340 [==============================] - 0s 74us/step - loss: 429019.3059 - acc: 0.0000e+00 - val_loss: 1181246.4967 - val_acc: 0.0000e+00
Epoch 320/1000
340/340 [==============================] - 0s 76us/step - loss: 390352.9114 - acc: 0.0000e+00 - val_loss: 989213.3618 - val_acc: 0.0000e+00
Epoch 321/1000
340/340 [==============================] - 0s 79us/step - loss: 456759.1292 - acc: 0.0000e+00 - val_loss: 1235997.4112 - val_acc: 0.0000e+00
Epoch 322/1000
340/340 [==============================] - 0s 76us/step - loss: 399998.5971 - acc: 0.0029 - val_loss: 1069568.1414 - val_acc: 0.0000e+00
Epoch 323/1000
340/340 [==============================] - 0s 76us/step - 

In [54]:
y_predict = model.predict(x_test)

In [55]:
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict): # 실제 정답값, 모델을 통한 예측값 
    return np.sqrt(mean_squared_error(y_test, y_predict))
print('RMSE :', RMSE(y_test, y_predict))

RMSE : 935.6625584333108


In [56]:
print(model.predict(final_predict_data))

[[56793.97]]
